In [1]:
# Add "parameters" (jupyter-notebook) tag to this cell, to allow papermill to inject different parameters
from datetime import date
it=0  #Iteration of gridsearch
# to put it all in one folder by date, will be replaced by papermill
today=date.today() 
rdate=today.strftime("%Y-%m-%d")
rdate="2021-08-31"
# Training Params
best_runs = []
training_stats = {}
epochs = 15
loops = 5

In [2]:
# Parameters
it = 7
rdate = "2021-08-31"


In [3]:
# See if running on Colab (for setting the correct workdir and installing all dependencies)
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  g_colab = True
else:
  print('Not running on CoLab')
  g_colab = False

Not running on CoLab


In [4]:
# connect to drive
if g_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/1 Job/Product and Code/CogAlex 2.0/"

# Choose model
# Gridsearch parameters
from sklearn.model_selection import ParameterGrid

# Original results with "xlm_roberta_base"
grid = [{"model_name": ["xlm-roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new"], ["de_train_new"], ["de_val_new"],
                      ["en_train_new", "en_val_new"], ["en_train_new"], ["en_val_new"], 
                      ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]},
        {"model_name": ["xlm-roberta-large", "distilbert-base-multilingual-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased", "roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]}]

pg = list(ParameterGrid(grid))
print(len(pg))
model_name = pg[it]["model_name"]
datasets = pg[it]["datasets"]
print(pg[it])

18
{'datasets': ['de_train_new', 'de_val_new', 'en_train_new', 'en_val_new'], 'model_name': 'xlm-roberta-base'}


# Manually set parameters

In [5]:
# Savedir
import os
savedir = f"./{rdate}/averages/{model_name}_{it}_{datasets}"
if g_colab:
    workdir = os.getcwd()
else:
    workdir = os.getcwd().replace("/home/","/binfl/")
model_dir = f"{workdir}/saved_models/{rdate}"

# Check for GPU support
import torch
if torch.cuda.is_available():
        print("Using GPU for inference")
        print(torch.cuda.get_device_name(torch.cuda.current_device()))
        device = torch.cuda.current_device()
else:
        print("Using CPU for inference")
        device = -1

Using GPU for inference
GeForce GTX 1080


In [6]:
if g_colab:
    !pip install transformers
    !pip install sentencepiece

# Libraries

In [7]:
import torch                                              #for training the model
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd                                       #for handling the data
from transformers import XLMRobertaTokenizer, AutoTokenizer              #for loading the pretrained model and tokenizer
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from sklearn import preprocessing                         #for label encoding
from sklearn.metrics import classification_report         #for showing performance on validation/test sets
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid         #for grid search
from sklearn.model_selection import ParameterSampler      #for random search
from sklearn.utils.fixes import loguniform
import sentencepiece
import matplotlib.pyplot as plt
import time
import datetime
import random
import numpy as np
%matplotlib inline
import seaborn as sns
# For file saving etc.
import os
import shutil

# Load Data

In [8]:
# sk learn label encoder for changing the labels to integers
labels=["ANT", "HYP", "RANDOM", "SYN"]
le = preprocessing.LabelEncoder()
le.fit(labels)
print(labels)

['ANT', 'HYP', 'RANDOM', 'SYN']


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
if "gpt2" in model_name:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token_id

1

# Evaluate Validation

In [10]:
label_dict = {i : l for i,l in enumerate(labels)}
print(label_dict)

{0: 'ANT', 1: 'HYP', 2: 'RANDOM', 3: 'SYN'}


In [11]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "all":0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "all":""}
for run in os.listdir(model_dir):
    print(run + "\n")
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",
                                                               num_labels=4,  
                                                               id2label=label_dict)                                                        
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    validgold_zh, validgold_de, validgold_en = [open(f"./datasets/validgold_{l}_data.txt").readlines() for l in ["chinese", "german", "english"]]
    validgold_all =   validgold_zh + validgold_de + validgold_en 
    for data, read_lang in zip([validgold_zh, validgold_de, validgold_en, validgold_all], ["chinese", "german", "english", "all"]):
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict

Run 3



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.85      0.87      0.86       136
         HYP       0.86      0.86      0.86       145
      RANDOM       0.94      0.94      0.94       428
         SYN       0.81      0.81      0.81       129

    accuracy                           0.89       838
   macro avg       0.86      0.87      0.87       838
weighted avg       0.89      0.89      0.89       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.90      0.87      0.88       136
         HYP       0.92      0.86      0.89       145
         SYN       0.87      0.81      0.84       129

   micro avg       0.90      0.84      0.87       410
   macro avg       0.90      0.84      0.87       410
weighted avg       0.90      0.84      0.87       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.50      0.43      0.46       275
         HYP       0.42      0.35      0.38       294
      RANDOM       0.72      0.77      0.74       786
         SYN       0.39      0.45      0.42       272

    accuracy                           0.58      1627
   macro avg       0.51      0.50      0.50      1627
weighted avg       0.57      0.58      0.58      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.43      0.53       275
         HYP       0.55      0.35      0.43       294
         SYN       0.48      0.45      0.46       272

   micro avg       0.56      0.41      0.47       841
   macro avg       0.58      0.41      0.47       841
weighted avg       0.58      0.41      0.47       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.58      0.50      0.53       308
         HYP       0.47      0.49      0.48       292
      RANDOM       0.78      0.81      0.79       877
         SYN       0.43      0.42      0.43       259

    accuracy                           0.64      1736
   macro avg       0.56      0.56      0.56      1736
weighted avg       0.64      0.64      0.64      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.76      0.50      0.60       308
         HYP       0.59      0.49      0.54       292
         SYN       0.51      0.42      0.46       259

   micro avg       0.62      0.47      0.54       859
   macro avg       0.62      0.47      0.53       859
weighted avg       0.63      0.47      0.54       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.61      0.54      0.57       719
         HYP       0.53      0.51      0.52       731
      RANDOM       0.79      0.82      0.80      2091
         SYN       0.48      0.51      0.50       660

    accuracy                           0.67      4201
   macro avg       0.60      0.59      0.60      4201
weighted avg       0.67      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.78      0.54      0.64       719
         HYP       0.66      0.51      0.57       731
         SYN       0.57      0.51      0.54       660

   micro avg       0.66      0.52      0.58      2110
   macro avg       0.67      0.52      0.58      2110
weighted avg       0.67      0.52      0.58      2110

Run 5



...

Results for all: 

              precision    recall  f1-score   support

         ANT       0.61      0.54      0.57       719
         HYP       0.53      0.51      0.52       731
      RANDOM       0.79      0.82      0.80      2091
         SYN       0.48      0.51      0.50       660

    accuracy                           0.67      4201
   macro avg       0.60      0.59      0.60      4201
weighted avg       0.67      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.78      0.54      0.64       719
         HYP       0.66      0.51      0.57       731
         SYN       0.57      0.51      0.54       660

   micro avg       0.66      0.52      0.58      2110
   macro avg       0.67      0.52      0.58      2110
weighted avg       0.67      0.52      0.58      2110

Run 1



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.88      0.88      0.88       136
         HYP       0.91      0.87      0.89       145
      RANDOM       0.93      0.95      0.94       428
         SYN       0.87      0.84      0.86       129

    accuracy                           0.91       838
   macro avg       0.90      0.89      0.89       838
weighted avg       0.91      0.91      0.91       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.92      0.88      0.89       136
         HYP       0.96      0.87      0.91       145
         SYN       0.92      0.84      0.88       129

   micro avg       0.93      0.86      0.90       410
   macro avg       0.93      0.86      0.90       410
weighted avg       0.93      0.86      0.90       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.45      0.40      0.42       275
         HYP       0.42      0.32      0.37       294
      RANDOM       0.72      0.78      0.75       786
         SYN       0.39      0.45      0.42       272

    accuracy                           0.58      1627
   macro avg       0.50      0.49      0.49      1627
weighted avg       0.57      0.58      0.57      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.62      0.40      0.49       275
         HYP       0.54      0.32      0.40       294
         SYN       0.48      0.45      0.46       272

   micro avg       0.54      0.39      0.45       841
   macro avg       0.55      0.39      0.45       841
weighted avg       0.55      0.39      0.45       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.56      0.51      0.54       308
         HYP       0.47      0.46      0.47       292
      RANDOM       0.77      0.81      0.79       877
         SYN       0.44      0.41      0.43       259

    accuracy                           0.64      1736
   macro avg       0.56      0.55      0.55      1736
weighted avg       0.63      0.64      0.64      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.73      0.51      0.60       308
         HYP       0.59      0.46      0.52       292
         SYN       0.54      0.41      0.47       259

   micro avg       0.62      0.47      0.53       859
   macro avg       0.62      0.46      0.53       859
weighted avg       0.63      0.47      0.53       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.59      0.54      0.56       719
         HYP       0.55      0.49      0.51       731
      RANDOM       0.78      0.83      0.80      2091
         SYN       0.50      0.51      0.50       660

    accuracy                           0.67      4201
   macro avg       0.60      0.59      0.60      4201
weighted avg       0.66      0.67      0.67      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.74      0.54      0.62       719
         HYP       0.66      0.49      0.56       731
         SYN       0.59      0.51      0.55       660

   micro avg       0.66      0.51      0.58      2110
   macro avg       0.67      0.51      0.58      2110
weighted avg       0.67      0.51      0.58      2110



In [12]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results.loc[k] = temp_df.mean()
avg_val_results.update(avg_val_results.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results

,precision,recall,f1-score,support
chinese,89.5,89.5,89.5,838.0
german,57.3,58.0,57.5,1627.0
english,63.8,64.2,63.9,1736.0
all,66.5,66.9,66.6,4201.0


In [13]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results_no_random.loc[k] = temp_df.mean()
avg_val_results_no_random.update(avg_val_results_no_random.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results_no_random

,precision,recall,f1-score,support
chinese,90.5,84.8,87.6,410.0
german,57.1,40.4,46.9,841.0
english,62.8,47.1,53.7,859.0
all,67.1,51.8,58.3,2110.0


In [14]:
# save average scores
os.makedirs(savedir, exist_ok=True)
val_preds_savedir = savedir + "/val_preds"
os.makedirs(val_preds_savedir, exist_ok=True)
avg_val_results.to_csv(f"{savedir}/val_avg_{model_name}_{it}.csv")
avg_val_results_no_random.to_csv(f"{savedir}/val_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(val_preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        

# Evaluate on Gold

In [15]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "italian":0, "german NEW":0, "english NEW": 0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "italian":"", "german NEW":"", "english NEW": ""}
for run in os.listdir(model_dir):
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",  id2label=label_dict)
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    gold_zh, gold_de, gold_en, gold_it = [open(f"./datasets/gold_{l}_data.txt").readlines() for l in ["chinese", "german", "english", "italian"]]
    gold_de_new, gold_en_new = [open(f"./datasets/gold_{l}_data_new.txt").readlines() for l in ["german", "english"]]   
    for data, read_lang in zip([gold_zh, gold_de, gold_en, gold_it, gold_de_new, gold_en_new], ["chinese", "german", "english", "italian", "german NEW", "english NEW"]):
        # Load data for prediction/scoring
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        
        #Predict labels on test_set
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict
        #print(results_no_random[run][read_lang]["weighted avg"])

Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.88      0.92      0.90       142
         HYP       0.90      0.86      0.88       129
      RANDOM       0.96      0.97      0.96       445
         SYN       0.85      0.82      0.84       122

    accuracy                           0.92       838
   macro avg       0.90      0.89      0.90       838
weighted avg       0.92      0.92      0.92       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.92      0.92      0.92       142
         HYP       0.93      0.86      0.90       129
         SYN       0.88      0.82      0.85       122

   micro avg       0.91      0.87      0.89       393
   macro avg       0.91      0.87      0.89       393
weighted avg       0.91      0.87      0.89       393



Results for german: 

              precision    recall  f1-score   support

         ANT       0.51      0.44      0.48       281
         HYP       0.48      0.43      0.45       286
      RANDOM       0.75      0.77      0.76       796
         SYN       0.38      0.44      0.41       265

    accuracy                           0.60      1628
   macro avg       0.53      0.52      0.52      1628
weighted avg       0.60      0.60      0.60      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.69      0.44      0.54       281
         HYP       0.61      0.43      0.51       286
         SYN       0.48      0.44      0.46       265

   micro avg       0.58      0.44      0.50       832
   macro avg       0.59      0.44      0.50       832
weighted avg       0.60      0.44      0.50       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.56      0.53      0.55       306
         HYP       0.44      0.41      0.43       279
      RANDOM       0.79      0.83      0.81       887
         SYN       0.42      0.41      0.42       266

    accuracy                           0.64      1738
   macro avg       0.55      0.55      0.55      1738
weighted avg       0.64      0.64      0.64      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.53      0.61       306
         HYP       0.56      0.41      0.47       279
         SYN       0.50      0.41      0.45       266

   micro avg       0.59      0.46      0.51       851
   macro avg       0.59      0.45      0.51       851
weighted avg       0.59      0.46      0.51       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.37      0.36      0.37       144
         HYP       0.36      0.38      0.37       153
      RANDOM       0.80      0.62      0.70       523
         SYN       0.31      0.51      0.39       187

    accuracy                           0.52      1007
   macro avg       0.46      0.47      0.46      1007
weighted avg       0.58      0.52      0.54      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.56      0.36      0.44       144
         HYP       0.54      0.38      0.44       153
         SYN       0.47      0.51      0.49       187

   micro avg       0.51      0.43      0.46       484
   macro avg       0.52      0.42      0.46       484
weighted avg       0.52      0.43      0.46       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.55      0.52      0.53       281
         HYP       0.55      0.57      0.56       286
      RANDOM       0.79      0.79      0.79       796
         SYN       0.44      0.45      0.44       265

    accuracy                           0.65      1628
   macro avg       0.58      0.58      0.58      1628
weighted avg       0.65      0.65      0.65      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.72      0.52      0.60       281
         HYP       0.66      0.57      0.61       286
         SYN       0.56      0.45      0.49       265

   micro avg       0.65      0.51      0.57       832
   macro avg       0.65      0.51      0.57       832
weighted avg       0.65      0.51      0.57       832



...

Results for german: 

              precision    recall  f1-score   support

         ANT       0.50      0.45      0.47       281
         HYP       0.47      0.40      0.43       286
      RANDOM       0.74      0.78      0.76       796
         SYN       0.38      0.42      0.40       265

    accuracy                           0.60      1628
   macro avg       0.52      0.51      0.52      1628
weighted avg       0.59      0.60      0.59      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.66      0.45      0.53       281
         HYP       0.60      0.40      0.48       286
         SYN       0.47      0.42      0.44       265

   micro avg       0.57      0.42      0.48       832
   macro avg       0.58      0.42      0.49       832
weighted avg       0.58      0.42      0.49       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.55      0.56      0.55       306
         HYP       0.47      0.42      0.45       279
      RANDOM       0.79      0.84      0.81       887
         SYN       0.45      0.41      0.43       266

    accuracy                           0.66      1738
   macro avg       0.57      0.56      0.56      1738
weighted avg       0.65      0.66      0.65      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.69      0.56      0.62       306
         HYP       0.58      0.42      0.49       279
         SYN       0.53      0.41      0.46       266

   micro avg       0.61      0.47      0.53       851
   macro avg       0.60      0.46      0.52       851
weighted avg       0.60      0.47      0.53       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.34      0.38      0.36       144
         HYP       0.37      0.33      0.35       153
      RANDOM       0.77      0.64      0.70       523
         SYN       0.34      0.51      0.41       187

    accuracy                           0.53      1007
   macro avg       0.46      0.46      0.45      1007
weighted avg       0.57      0.53      0.54      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.59      0.38      0.46       144
         HYP       0.54      0.33      0.41       153
         SYN       0.47      0.51      0.49       187

   micro avg       0.52      0.41      0.46       484
   macro avg       0.54      0.41      0.45       484
weighted avg       0.53      0.41      0.46       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.56      0.53      0.55       281
         HYP       0.55      0.52      0.53       286
      RANDOM       0.79      0.80      0.80       796
         SYN       0.44      0.47      0.45       265

    accuracy                           0.65      1628
   macro avg       0.59      0.58      0.58      1628
weighted avg       0.65      0.65      0.65      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.72      0.53      0.61       281
         HYP       0.67      0.52      0.59       286
         SYN       0.53      0.47      0.50       265

   micro avg       0.64      0.51      0.57       832
   macro avg       0.64      0.51      0.57       832
weighted avg       0.64      0.51      0.57       832



Results for english NEW: 

              precision    recall  f1-score   support

         ANT       0.61      0.61      0.61       306
         HYP       0.56      0.55      0.56       283
      RANDOM       0.84      0.85      0.85       887
         SYN       0.53      0.52      0.52       262

    accuracy                           0.71      1738
   macro avg       0.64      0.63      0.63      1738
weighted avg       0.71      0.71      0.71      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.75      0.61      0.67       306
         HYP       0.67      0.55      0.61       283
         SYN       0.60      0.52      0.56       262

   micro avg       0.68      0.56      0.61       851
   macro avg       0.67      0.56      0.61       851
weighted avg       0.68      0.56      0.61       851



In [16]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results.loc[k] = temp_df.mean()
avg_test_results.update(avg_test_results.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results

,precision,recall,f1-score,support
chinese,92.2,92.2,92.2,838.0
german,59.9,60.0,59.8,1628.0
english,64.0,64.7,64.3,1738.0
italian,58.1,52.6,54.3,1007.0
german NEW,64.9,65.0,64.9,1628.0
english NEW,70.1,70.0,70.0,1738.0


In [17]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results_no_random.loc[k] = temp_df.mean()
avg_test_results_no_random.update(avg_test_results_no_random.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results_no_random

,precision,recall,f1-score,support
chinese,91.5,87.0,89.1,393.0
german,59.3,43.5,49.9,832.0
english,59.3,45.8,51.7,851.0
italian,52.0,42.3,46.0,484.0
german NEW,64.7,51.1,57.0,832.0
english NEW,67.7,55.3,60.7,851.0


In [18]:
# save average scores
os.makedirs(savedir, exist_ok=True)
preds_savedir = savedir + "/preds"
os.makedirs(preds_savedir, exist_ok=True)
avg_test_results.to_csv(f"{savedir}/test_avg_{model_name}_{it}.csv")
avg_test_results_no_random.to_csv(f"{savedir}/test_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        